In [1]:
import sparknlp
# spark = sparknlp.start() 
spark = sparknlp.start(apple_silicon=True)

from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import *
import pyspark
print(f"Spark NLP version: {sparknlp.version()}")
print(f"Spark version: {pyspark.__version__}")



25/12/16 11:19:10 WARN Utils: Your hostname, Dannys-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 100.116.131.14 instead (on interface utun4)
25/12/16 11:19:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/dannytram/MYSELF/MLOps_K6/final_project/Livestream-Assistant-Agent/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/dannytram/.ivy2/cache
The jars for the packages stored in: /Users/dannytram/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp-silicon_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-03df4810-ddc7-4d20-a414-5188135f8db6;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp-silicon_2.12;6.2.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath

25/12/16 11:19:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark NLP version: 6.2.3
Spark version: 3.3.1


In [2]:
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
)

dataset_path = "data/gold/training_data.parquet"

data_schema = StructType([
    StructField("reviewText", StringType(), True),
    StructField("semanticDetect", StringType(), True),
])

# Load the dataset
df = spark.read.schema(data_schema).parquet(dataset_path)

print(f"Total records: {df.count()}")
print(df.show(5))

# Split into train (70%) and test (30%)
train_df, test_df = df.randomSplit([0.7, 0.3], seed=42)

print(f"Train records: {train_df.count()}")
print(f"Test records: {test_df.count()}")

print(train_df.show(5))

Total records: 7037123
+--------------------+--------------+
|          reviewText|semanticDetect|
+--------------------+--------------+
|great quality and...|      positive|
|The case is nice ...|      positive|
|Doesn't fit my ph...|      negative|
|Haven't used it a...|      positive|
|I love these cove...|      positive|
+--------------------+--------------+
only showing top 5 rows

None


Train records: 4925844


Test records: 2111279
+--------------------+--------------+
|          reviewText|semanticDetect|
+--------------------+--------------+
|\nPurchased from ...|      negative|
|\nThese earphones...|      positive|
|  &nbsp;It is NOT...|      negative|
|  ,    ,       , ...|      positive|
|  ,   .Touch and ...|      positive|
+--------------------+--------------+
only showing top 5 rows

None


In [ ]:
MODEL_NAME = "sentimentdl_use_twitter"

documentAssembler = DocumentAssembler()\
    .setInputCol("reviewText")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
.setInputCols(["document"])\
.setOutputCol("sentence_embeddings")


sentimentdl = SentimentDLModel.pretrained(name=MODEL_NAME, lang="en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
    stages = [
        documentAssembler,
        use,
        sentimentdl
        ])

print("Pipeline defined successfully!")

tfhub_use download started this may take some time.
25/12/16 11:27:42 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_twitter download started this may take some time.
25/12/16 11:27:46 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
Approximate size to download 11.4 MB
[OK!]
Pipeline defined successfully!


In [7]:
# Note: We're not training anything, just fitting the pipeline structure
# This is required even for pre-trained models
print("\nFitting pipeline structure (no training involved)...")
pipelineModel = nlpPipeline.fit(test_df)
print("Pipeline ready for inference!")


Fitting pipeline structure (no training involved)...
Pipeline ready for inference!


In [ ]:
from pyspark.sql.functions import col

print("\n=== Running Sentiment Analysis on Test Set ===")
print("This may take a while for large datasets...")

# Transform the test data
results = pipelineModel.transform(test_df)

# Extract sentiment from the annotation structure
# Spark NLP stores results in an array of annotations
results_with_sentiment = results.withColumn(
    "predicted_sentiment", 
    col("sentiment.result").getItem(0)
)

print(results_with_sentiment.show())

# Show results
# print("\nSample predictions:")
results_with_sentiment.select("reviewText", "semanticDetect", "predicted_sentiment").show(10, truncate=50)



=== Running Sentiment Analysis on Test Set ===
This may take a while for large datasets...


+--------------------+--------------+--------------------+--------------------+--------------------+-------------------+
|          reviewText|semanticDetect|            document| sentence_embeddings|           sentiment|predicted_sentiment|
+--------------------+--------------+--------------------+--------------------+--------------------+-------------------+
|                   .|      positive|[{document, 0, 19...|[{sentence_embedd...|[{category, 0, 19...|           positive|
|           Awesome  |      positive|[{document, 0, 10...|[{sentence_embedd...|[{category, 0, 10...|           positive|
|               great|      positive|[{document, 0, 6,...|[{sentence_embedd...|[{category, 0, 6,...|           positive|
| -  -.   . .\n ( ...|      positive|[{document, 0, 72...|[{sentence_embedd...|[{category, 0, 72...|           positive|
| Easy to fit on d...|      positive|[{document, 0, 88...|[{sentence_embedd...|[{category, 0, 88...|           positive|
| F A N T A S T I ...|      posi